# Project 3 - Pacman

# Members:
RA 183374 - Helena Steck

RA 187251 - Tainá Turella C. dos Santos

## Introduction
During Project 3 - Pacman the group was required to apply its knowledge around genetic algorithms and reinforcement learning to teach Mr. Pacman to defeat the ghosts while he feeds himself in a maze.
This project was harder than the previous projects because we needed to comprehend how the codified game works and then implement the algorithms in a way that they could interact with each other without friction/issues.

## Code & Difficulties
We used the pacman implementation that is available at [Berkeley](https://inst.eecs.berkeley.edu/~cs188/sp19/assets/files/search.zip "Search.zip") and over this implementation we managed to train our model using reinforcement learning.
We were not able to comprehend how to implement the theory learned about genetic learning, therefore you won't be able to see information about genetic training in this report and we won't be able to make comparisons between the 2 models.

## Reinforcement Learning
The definition of Reinforcement Learning available in [Wikipedia](https://en.wikipedia.org/wiki/Reinforcement_learning "Reinforcement learning") is:
> *Reinforcement learning (RL) is an area of machine learning concerned with how intelligent agents ought to take actions in an environment in order to maximize the notion of cumulative reward.Reinforcement learning is one of three basic machine learning paradigms, alongside supervised learning and unsupervised learning.
Reinforcement learning differs from supervised learning in not needing labelled input/output pairs be presented, and in not needing sub-optimal actions to be explicitly corrected. Instead the focus is on finding a balance between exploration (of uncharted territory) and exploitation (of current knowledge).
The environment is typically stated in the form of a Markov decision process (MDP), because many reinforcement learning algorithms for this context use dynamic programming techniques.The main difference between the classical dynamic programming methods and reinforcement learning algorithms is that the latter do not assume knowledge of an exact mathematical model of the MDP and they target large MDPs where exact methods become infeasible.*

Our implementation will be based on those concepts using Q-Learning, which is a reinforcement learning algorithm that is based on knowing the value of an action given a particular state. The algorithim calculates the quality of a state-action combination and after each iteration the weigth of each feature is updated and at the end of the training the converged weights are used as params for the Agents.
Using the Aproximate Q-Learning we tried to fit reward function as a linear function that is obtained due to the combination of the features applied and analysed in the model.
One thing that the group believe that should be discussed is the impact that biases can cause in reinforcement learning algorithms. For example, in this project was very tough to teach pacman to eat the ghosts when they are "scared" that happens because in other point it is crucial for pacman to run away from them when they can be harmfull. This is just a silly example, but at the same time made us question somethings during the development.
There are some advantages on using Aproximate Q-Learning, but there are also some disadvantages, such as; the restriction that it applies to the accuracy of learned rewards and it also requires well-known features. It is a more generic algorithm implementation, but its also not perfect.

### Small Grid Training
### Medium Grid Training
### Classic Grid Training

## Disclaimer:
This project was equally divided between the two group members. Therefore we confirm that the methods, the discussions and the report elaboration were created by both of us. We used the code available at [Berkeley](https://inst.eecs.berkeley.edu/~cs188/sp19/assets/files/search.zip "Search.zip") to run pacman and we used their tutorial as a guide on how to implement the methods of Reinforcement Learning.

We did the entire project during calls via Google Meets, using a pair programming method. In that way we could help each other during the development phase.